In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

llm = ChatOpenAI(model="gpt-4o-mini")

for c in llm.stream([HumanMessage("잘 지냈어? 한국 사회의 문제점이 무엇인지 이야기해줘.")]):
    print(c.content, end='|')

|안|녕하세요|!| 저는| 잘| 지|내|고| 있어|요|.| 한국| 사회|의| 문제|점|은| 여러| 가지|가| 있지만|,| 몇| 가지| 주요|한| 이|슈|를| 이야기|해|볼|게|요|.

|1|.| **|고|령|화| 사회|**|:| 한국|은| 급|속|한| 고|령|화|가| 진행|되고| 있습니다|.| 이는| 연|금| 시스템|,| 의료| 서비스|,| 일|자리| 문제| 등| 다양한| 사회|적| 문제|를| 초|래|하고| 있습니다|.

|2|.| **|청|년| 실|업|**|:| 높은| 학|력|에도| 불|구|하고| 청|년|층|의| 실|업|률|이| 지속|적으로| 높은| 상황|입니다|.| 일|자리| 부족|과| 경쟁| 심|화|로| 인해| 많은| 청|년|들이| 어려|움을| 겪|고| 있습니다|.

|3|.| **|주|거| 문제|**|:| 주|택| 가격| 상승|과| 전|세|난| 등|으로| 인해| 많은| 사람들이| 안정|된| 주|거| 환경|을| 갖|기| 어려|운| 상황|입니다|.| 특히| 서울|과| 같은| 대|도|시|에서| 이| 문제가| 두|드|러|집|니다|.

|4|.| **|정|신| 건강| 문제|**|:| 경쟁|과| 스트|레스|가| 만|연|한| 사회|에서| 정신| 건강| 문제가| 증가|하고| 있습니다|.| 이에| 대한| 인|식| 부족|과| 지원| 체|계| 부족|도| 문제|로| 지|적|되고| 있습니다|.

|5|.| **|성| 평|등| 문제|**|:| 성|별| 불|균|형|,| 직|장에서|의| 성|차|별| 등| 여|전히| 성| 평|등|이| 이|뤄|지|지| 않는| 상황|입니다|.| 또한|,| 성|범|죄| 문제|도| 심|각|하며| 이에| 대한| 공|론|화|가| 필요|합니다|.

|6|.| **|교육| 경쟁|**|:| 지나|치|게| 치|열|한| 교육| 경쟁|이| 학생|들에게| 심|리|적| 압|박|을| 주|고|,| 다양한| 교육|적| 접근|이| 실패|하고| 있는| 상황|입니다|.

|이| 외|에도| 다양한| 이|슈|가| 있지만|,| 사회|적| 논|의|와| 해결|책| 마련|이| 필요한| 부분|들이

In [3]:
from langchain_core.tools import tool
from datetime import datetime
import pytz

@tool # @tool 데코레이터를 사용하여 함수를 도구로 등록
def get_current_time(timezone: str, location: str) -> str:
    """ 현재 시각을 반환하는 함수

    Args:
        timezone (str): 타임존 (예: 'Asia/Seoul') 실제 존재하는 타임존이어야 함
        location (str): 지역명. 타임존이 모든 지명에 대응되지 않기 때문에 이후 llm 답변 생성에 사용됨
    """
    tz = pytz.timezone(timezone)
    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    location_and_local_time = f'{timezone} ({location}) 현재시각 {now} ' # 타임존, 지역명, 현재시각을 문자열로 반환
    print(location_and_local_time)
    return location_and_local_time

# 도구를 tools 리스트에 추가하고, tool_dict에도 추가
tools = [get_current_time,]
tool_dict = {"get_current_time": get_current_time,}

# 도구를 모델에 바인딩: 모델에 도구를 바인딩하면, 도구를 사용하여 llm 답변을 생성할 수 있음
llm_with_tools = llm.bind_tools(tools)

In [4]:
from langchain_core.messages import SystemMessage

messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage("부산은 지금 몇 시야?"),
]

response = llm_with_tools.stream(messages)

is_first = True
for chunk in response:
    print("chunk type: ", type(chunk))

    if is_first:
        is_first = False
        gathered = chunk
    else:
        gathered += chunk

    print("content: ", gathered.content, "tool_call_chunk", gathered.tool_calls)

messages.append(gathered)

chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_SrLaakOJZOIrecHPGLaO3Q9n', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_SrLaakOJZOIrecHPGLaO3Q9n', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_SrLaakOJZOIrecHPGLaO3Q9n', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {'timezone': ''}, 'id': 'call_SrLaakOJZOIrecHPGLaO3Q9n', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {'timezone': 'Asia'}, 'id': 'call_SrLaakOJZOIrecHPGLaO3Q9n', 'type': 'tool_c

In [5]:
gathered

AIMessageChunk(content='', additional_kwargs={}, response_metadata={'model_provider': 'openai', 'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_1590f93f9d', 'service_tier': 'default'}, id='lc_run--019c0aa7-910c-7fb0-8bb6-09208f33898c', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '부산'}, 'id': 'call_SrLaakOJZOIrecHPGLaO3Q9n', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 135, 'output_tokens': 23, 'total_tokens': 158, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}, tool_call_chunks=[{'name': 'get_current_time', 'args': '{"timezone":"Asia/Seoul","location":"부산"}', 'id': 'call_SrLaakOJZOIrecHPGLaO3Q9n', 'index': 0, 'type': 'tool_call_chunk'}], chunk_position='last')

In [8]:
for tool_call in gathered.tool_calls:
    selected_tool = tool_dict[tool_call["name"]]  # tool_dict를 사용하여 도구 이름으로 도구 함수 선택
    print(tool_call["args"])  # 도구 호출 시 전달된 인자 출력
    tool_msg = selected_tool.invoke(tool_call)  # 도구 함수를 호출하여 결과 반환
    messages.append(tool_msg)

messages

{'timezone': 'Asia/Seoul', 'location': '부산'}
Asia/Seoul (부산) 현재시각 2026-01-30 23:38:29 


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산은 지금 몇 시야?', additional_kwargs={}, response_metadata={}),
 AIMessageChunk(content='', additional_kwargs={}, response_metadata={'model_provider': 'openai', 'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_1590f93f9d', 'service_tier': 'default'}, id='lc_run--019c0aa7-910c-7fb0-8bb6-09208f33898c', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '부산'}, 'id': 'call_SrLaakOJZOIrecHPGLaO3Q9n', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 135, 'output_tokens': 23, 'total_tokens': 158, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}, tool_call_chunks=[{'name': 'get_current_time', 'args': '{"timezone":"Asia/Seoul","location":"부산"}', 'id': 'call_SrLaakOJZOIrecHPGLaO3Q9n', 'index': 0, 't

In [9]:
for c in llm_with_tools.stream(messages):
    print(c.content, end='|')

|부|산|은| 현재| |202|6|년| |1|월| |30|일| |23|시| |38|분|입니다|.||||